##  Sentinel-3 SLSTR composites

### Service Definition

In [ ]:
service = dict([('title', 'Sentinel-3 SLSTR composites'),
                ('abstract', 'Sentinel-3 SLSTR composites for descending acquisitions'),
                ('id', 'ewf-s3-slstr-composites')])

### Parameter Definition 

In [ ]:
false_color_infrared = dict([('id', 'false_color_infrared'),
                       ('title', 'False Color Infrared (S3, S2, S1)'),
                       ('abstract', 'False Color Infrared (S3, S2, S1)'),
                       ('value', 'Yes'),
                       ('options', 'Yes,No')])

In [ ]:
false_color_1 = dict([('id', 'false_color_1'),
                       ('title', 'False Color 1 (S8, S1, S5)'),
                       ('abstract', 'False Color 1 (S8, S1, S5)'),
                       ('value', 'Yes'),
                       ('options', 'Yes,No')])

In [ ]:
false_color_2 = dict([('id', 'false_color_2'),
                       ('title', 'False Color 2 (S5, S3, S2)'),
                       ('abstract', 'False Color 2 (S5, S3, S2)'),
                       ('value', 'Yes'),
                       ('options', 'Yes,No')])

### Runtime parameter definition

**Input reference**

The input identifier is the catalogue entry URL (a.k.a. self value).

In [ ]:
input_reference = dict([('identifier', 'input_reference'),
                        ('title', 'Sentinel-3 SLSTR Level-1 (SL_1_RBT___ descending pass)'),
                        ('abstract', 'This service takes as input a Sentinel-3 SLSTR Level 1 (SL_1_RBT___) product on DESCENDING pass and does False Colour RGB composites'),
                        ('value', 'https://catalog.terradue.com/sentinel3/search?uid=S3B_SL_1_RBT____20190522T044019_20190522T044319_20190523T100228_0179_025_304_2520_LN2_O_NT_003')])

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data'

### Workflow

#### Import the packages

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.environ['PREFIX'], 'conda-otb/lib/python'))
os.environ['OTB_APPLICATION_PATH'] = os.path.join(os.environ['PREFIX'], 'conda-otb/lib/otb/applications')
os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
import otbApplication
import gdal
from helpers import *
import geopandas as gp
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import box
import cioppy
import shutil
gdal.UseExceptions()

In [ ]:
%load_ext autoreload
%autoreload 2

### Get product metadata

In [ ]:
ciop = cioppy.Cioppy()

In [ ]:
input_metadata = gp.GeoDataFrame(ciop.search(end_point=input_reference['value'], 
                                 params=[],
                                 output_fields='identifier,self,wkt,startdate,enddate,enclosure,orbitDirection', 
                                 model='EOP'))


input_metadata['geometry'] = input_metadata['wkt'].apply(loads)
input_metadata['startdate'] = pd.to_datetime(input_metadata['startdate'])
input_metadata['enddate'] = pd.to_datetime(input_metadata['enddate'])

In [ ]:
input_metadata

In [ ]:
if input_metadata.iloc[0]['orbitDirection'] != 'DESCENDING':
    ciop.log('ERROR','Product cannot be used as input')
    raise Exception('Only use products with Orbit=Descending')

### Import Sentinel-3 SLSTR product

In [ ]:
operators = ['Read', 
             'Rad2Refl',
             'Resample',
             'Reproject',
             'Write']

In [ ]:
read = dict()

s3_path = '{0}/{1}/{1}.SEN3/xfdumanifest.xml'.format(data_path, input_metadata.iloc[0]['identifier'])

read['file'] =  s3_path
read['formatName'] = 'Sen3_SLSTRL1B_500m'

rad2refl = dict()

rad2refl['sensor'] = 'SLSTR_500m'
rad2refl['copyTiePointGrids'] = 'true'
rad2refl['copyFlagBandsAndMasks'] = 'true'
rad2refl['copyNonSpectralBands'] = 'true'

resample = dict()
resample['referenceBandName'] = 'F1_BT_in'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

write = dict()
write['file'] = 's3_slstr'

In [ ]:
snap_graph(os.environ['GPT_BIN'],
           operators,
              Read=read, 
              Rad2Refl=rad2refl,
              Resample=resample,
              Reproject=reproject,
              Write=write)

### RGB Composites

In [ ]:
output_startdate = input_metadata.iloc[0]['startdate']
output_stopdate = input_metadata.iloc[0]['enddate']

In [ ]:
composites = dict()

composites['S3 SLSTR False color Infrared'] = {'bands': 'S3_reflectance_an,S2_reflectance_an,S1_reflectance_an',
                                               'create': True if (false_color_infrared['value'] == 'Yes') else False,
                                               'hfact': 3.0}

composites['S3 SLSTR False color 1'] = {'bands': 'S8_BT_in,S1_reflectance_an,S5_reflectance_an',
                                               'create': True if (false_color_1['value'] == 'Yes') else False,
                                               'hfact': 3.0}

composites['S3 SLSTR False color 2'] = {'bands': 'S5_reflectance_an,S3_reflectance_an,S2_reflectance_an',
                                               'create': True if (false_color_2['value'] == 'Yes') else False,
                                               'hfact': 3.0}

In [ ]:
for k, v in composites.items():
    
    print(k)
    
    bands = [os.path.join(write['file'] + '.data',  '{}.img'.format(band)) for band in (composites[k]['bands'].split(',') +  
                                                                         ['cloud_an',
                                                                          'confidence_an',
                                                                          'S2_exception_an'])]
    
    print(bands)
    
    ds = gdal.Open(bands[0])

    geo_transform = ds.GetGeoTransform()
    projection_ref = ds.GetProjectionRef()
    
    ds = None
    
    s3_rgb_data = read_s3(bands)
    
    red = s3_rgb_data[:,:,0]
    green = s3_rgb_data[:,:,1]
    blue = s3_rgb_data[:,:,2]
    cloud = s3_rgb_data[:,:,3]
    confidence = s3_rgb_data[:,:,4]
    exception = s3_rgb_data[:,:,5]

    date_format = '%Y%m%dT%H%m%S'
    
    output_name = '-'.join([k.replace(' ', '-').upper(), 
                            output_startdate.strftime(date_format), 
                            output_startdate.strftime(date_format)])
    
    s3_rgb_composite(red, 
                     green,
                     blue, 
                     cloud, 
                     confidence,
                     exception,
                     geo_transform,
                     projection_ref, 
                     output_name + '.tif')
    
     # PNG
    gdal.Translate('{}.png'.format(output_name), 
                   '{}.tif'.format(output_name), 
                   format='PNG')

    os.remove('{}.png.aux.xml'.format(output_name))
    
    
    date_format = '%Y-%m-%dT%H:%m:%S'
    
    with open(output_name + '.tif.properties', 'w') as file:
        file.write('title={} ({}/{})\n'.format(k, 
                                               output_startdate.strftime(date_format),
                                               output_stopdate.strftime(date_format)))

        file.write('date={}Z/{}Z\n'.format(output_startdate.strftime(date_format),
                                               output_stopdate.strftime(date_format)))   
        file.write('geometry={}'.format(input_metadata.iloc[0].wkt))

   

    with open(output_name + '.png.properties', 'w') as file:
        file.write('title={} - Quicklook ({}/{})\n'.format(k, 
                                               output_startdate.strftime(date_format),
                                               output_stopdate.strftime(date_format)))

        file.write('date={}Z/{}Z\n'.format(output_startdate.strftime(date_format),
                                           output_stopdate.strftime(date_format)))   
        file.write('geometry={}'.format(input_metadata.iloc[0].wkt))

### Clean-up

In [ ]:
shutil.rmtree('{}.data'.format(write['file']))

In [ ]:
os.remove('{}.dim'.format(write['file']))

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.